In [65]:
import pyblp
import numpy as np
import pandas as pd
import random

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.12.0'

In [2]:
product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
product_data.head()

,market_ids,clustering_ids,car_ids,firm_ids,region,shares,prices,hpwt,air,mpd,...,supply_instruments2,supply_instruments3,supply_instruments4,supply_instruments5,supply_instruments6,supply_instruments7,supply_instruments8,supply_instruments9,supply_instruments10,supply_instruments11
0,1971,AMGREM71,129,15,US,0.001051,4.935802,0.528997,0,1.888146,...,0.0,1.705933,1.595656,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.888146
1,1971,AMHORN71,130,15,US,0.000670,5.516049,0.494324,0,1.935989,...,0.0,1.680910,1.490295,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.935989
2,1971,AMJAVL71,132,15,US,0.000341,7.108642,0.467613,0,1.716799,...,0.0,1.801067,1.357703,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.716799
3,1971,AMMATA71,134,15,US,0.000522,6.839506,0.426540,0,1.687871,...,0.0,1.818061,1.261347,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.687871
4,1971,AMAMBS71,136,15,US,0.000442,8.928395,0.452489,0,1.504286,...,0.0,1.933210,1.237365,87.0,-61.959985,0.0,46.060389,29.786989,0.0,1.504286


In [3]:
agent_data = pd.read_csv(pyblp.data.BLP_AGENTS_LOCATION)
agent_data.head()

,market_ids,weights,nodes0,nodes1,nodes2,nodes3,nodes4,income
0,1971,0.000543,1.192188,0.478777,0.980830,-0.824410,2.473301,109.560369
1,1971,0.000723,1.497074,-2.026204,-1.741316,1.412568,-0.747468,45.457314
2,1971,0.000544,1.438081,0.813280,-1.749974,-1.203509,0.049558,127.146548
3,1971,0.000701,1.768655,-0.177453,0.286602,0.391517,0.683669,22.604045
4,1971,0.000549,0.849970,-0.135337,0.735920,1.036247,-1.143436,170.226032


In [4]:
product_formulations = (
   pyblp.Formulation('1 + hpwt + air + mpd + space'),
   pyblp.Formulation('1 + prices + hpwt + air + mpd + space'),
   pyblp.Formulation('1 + log(hpwt) + air + log(mpg) + log(space) + trend')
)
product_formulations

(1 + hpwt + air + mpd + space,
 1 + prices + hpwt + air + mpd + space,
 1 + log(hpwt) + air + log(mpg) + log(space) + trend)

In [5]:
agent_formulation = pyblp.Formulation('0 + I(1 / income)')
agent_formulation

I(1 / income)

In [7]:
problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data, costs_type='log')
problem

Dimensions:
 T    N     F    I     K1    K2    K3    D    MD    MS 
---  ----  ---  ----  ----  ----  ----  ---  ----  ----
20   2217  26   4000   5     6     6     1    13    18 

Formulations:
       Column Indices:            0          1       2       3          4         5  
-----------------------------  --------  ---------  ----  --------  ----------  -----
 X1: Linear Characteristics       1        hpwt     air     mpd       space          
X2: Nonlinear Characteristics     1       prices    hpwt    air        mpd      space
X3: Log Cost Characteristics      1      log(hpwt)  air   log(mpg)  log(space)  trend
       d: Demographics         1/income                                              

In [181]:
initial_sigma = np.diag([3.612, 0, 4.628, 1.818, 1.050, 2.056])
initial_pi = np.c_[[0, -43.501, 0, 0, 0, 0]]

In [9]:
results = problem.solve(
    initial_sigma,
    initial_pi,
    costs_bounds=(0.001, None),
    W_type='clustered',
    se_type='clustered',
    initial_update=True,
)
results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares    Costs   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  -------  ----------------  -----------------
 2    +5.0E+02     +6.2E-06        +4.9E-01         +5.1E+02         0        0         +4.2E+09          +3.8E+08     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:05:17       No           57           111         33450       102517   

Nonlinear Coefficient Estimates (Robust SEs Adjusted for 999 Clusters in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |

In [122]:
resultsSet = [type(results)]*26
initial_sigma_Set = [type(initial_sigma)]*26
initial_pi_Set = [type(initial_pi)]*26


#save the original results to resultsSet as the last data
resultsSet[25] = results
initial_sigma_Set[25] = initial_sigma
initial_pi_Set[25] = initial_pi

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares    Costs   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  -------  ----------------  -----------------
 2    +5.0E+02     +6.2E-06        +4.9E-01         +5.1E+02         0        0         +4.2E+09          +3.8E+08     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:05:17       No           57           111         33450       102517   

Nonlinear Coefficient Estimates (Robust SEs Adjusted for 999 Clusters in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |

In [124]:
for i in range (25):## from 0 to 24
    x1 = 1+random.uniform(-0.2, 0.2)
    x2 = 1+random.uniform(-0.2, 0.2)
    x3 = 1+random.uniform(-0.2, 0.2)
    x4 = 1+random.uniform(-0.2, 0.2)
    x5 = 1+random.uniform(-0.2, 0.2)
    x6 = 1+random.uniform(-0.2, 0.2)
    
    initial_sigma_Set[i] = np.diag([3.612*x1, 0, 4.628*x2, 1.818*x3, 1.050*x4, 2.056*x5])
    initial_pi_Set[i] = np.c_[[0, -43.501*x6, 0, 0, 0, 0]]
    
    resultsSet[i] = problem.solve(
        initial_sigma_Set[i],
        initial_pi_Set[i],
        costs_bounds=(0.001, None),
        W_type='clustered',
        se_type='clustered',
        initial_update=True,
    )

In [203]:
index = ['']*26
ComTime = ['']*26
Converge = ['']*26
ObjectEvaluations = ['']*26
pi = ['']*26
initial_sigmas = ['']*26
initial_pis = ['']*26
betas = ['']*26
gammas = ['']*26

for i in range (26): ## 0 - 25
    index[i] = i
    ComTime [i] = resultsSet[i].total_time #computation time,
    Converge[i] = resultsSet[i].converged #optimizer converged,
    ObjectEvaluations[i] = resultsSet[i].objective_evaluations #objective evaluations 
    pi[i] = resultsSet[i].pi[1] # Pi for price
    betas[i] = resultsSet[i].beta # beta estimates
    gammas[i] = resultsSet[i].gamma # gamma estimates
    initial_sigmas[i] = initial_sigma_Set[i].diagonal() #initial sigma used in this iteration (only report the diagonal)
    initial_pis[i] = initial_pi_Set[i][1]  #initial pi used in this iteration (only report the nonzero values)
    

df26 = pd.DataFrame({'index': index,
                    'initial_sigmas':initial_sigmas,
                    'initial_pis': initial_pis,
                    'ComTime': ComTime,
                    'Converge': Converge,
                    'ObjectEvaluations':ObjectEvaluations,
                    'pi': pi,
                    'betas': betas,
                    'gammas': gammas})

SyntaxError: can't assign to function call (<ipython-input-203-696ab453e71b>, line 34)

In [184]:
df26.to_csv('df26.csv')